In [1]:
import pandas as pd 
import datetime

In [2]:
tripleg = pd.read_csv('triplegs.csv')

In [3]:
food_df = pd.read_csv('food_inference.csv', index_col=0)

In [4]:
food_df.drop(columns=['id', 'finished_at'], inplace=True)
food_df

,user_id,started_at,lat,lon,retail_id,retail_lat,retail_lon
0,1,2022-09-23 12:16:32-04:00,30.167915,-81.631710,168227,30.167103,-81.631638
1,1,2022-09-24 20:52:52-04:00,30.146970,-81.632480,151588,30.147409,-81.632477
2,1,2022-10-10 19:04:37-04:00,30.122375,-81.625240,151863,30.121626,-81.625488
3,4,2022-09-06 13:17:00-04:00,30.320775,-81.489335,217378,30.321608,-81.489418
4,4,2022-09-08 01:41:12-04:00,30.321386,-81.494801,167193,30.320749,-81.495316
...,...,...,...,...,...,...,...
17205,12366,2022-10-28 17:49:28-04:00,30.391939,-81.730192,14499,30.391975,-81.730347
17206,12380,2022-10-29 22:07:01-04:00,30.172684,-81.604192,32978,30.172754,-81.603531
17207,12388,2022-10-27 19:34:13-04:00,30.268400,-81.613899,237958,30.268553,-81.613953
17208,12388,2022-10-31 19:12:20-04:00,30.269813,-81.610858,45499,30.270267,-81.610809


In [5]:
tripleg.drop(columns=['id'], inplace=True)
tripleg

,user_id,started_at,finished_at,geom
0,0,2022-09-09 01:01:29-04:00,2022-09-09 01:17:29-04:00,LINESTRING (-81.5065289500000034 30.2162162199...
1,1,2022-09-05 17:52:08-04:00,2022-09-05 17:52:57-04:00,LINESTRING (-81.6268599999999935 30.1161999999...
2,1,2022-09-05 20:41:05-04:00,2022-09-05 20:42:55-04:00,LINESTRING (-81.7111000000000018 30.1017399999...
3,1,2022-09-05 21:38:31-04:00,2022-09-05 21:38:42-04:00,LINESTRING (-81.8334399999999960 30.1061100000...
4,1,2022-09-05 23:31:17-04:00,2022-09-05 23:31:28-04:00,LINESTRING (-81.7179699999999940 30.1011300000...
...,...,...,...,...
413238,12419,2022-10-23 17:03:18-04:00,2022-10-23 17:03:30-04:00,LINESTRING (-81.6054400000000015 30.3792200000...
413239,12419,2022-10-23 17:41:20-04:00,2022-10-23 17:41:30-04:00,LINESTRING (-81.6054306099999991 30.3792133299...
413240,12419,2022-10-26 20:13:01-04:00,2022-10-26 20:18:58-04:00,LINESTRING (-81.6119800000000026 30.3824900000...
413241,12419,2022-10-27 01:06:08-04:00,2022-10-27 01:09:53-04:00,LINESTRING (-81.6119800000000026 30.3824900000...


In [6]:
food_df['started_at'] = pd.to_datetime(food_df['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [7]:
ID_list=food_df['user_id'].unique()
len(ID_list)

1804

In [8]:
time_window = datetime.timedelta(minutes=2)

miss = 0
is_unique_check = 0
data_to_append_list = []
for id in ID_list:
    related_food_by_user = food_df[food_df['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for food_related_row in related_food_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > food_related_row[2] - time_window) &
            (related_tripled_by_user['finished_at'] <= food_related_row[2])
        ]
        if len(tripled_found) == 0: continue #did not found related trip
        if len(tripled_found) > 0:
            is_unique_check += 1
            
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'lat': food_related_row[3], 'lon': food_related_row[4],
                'retail_id': food_related_row[5], 'retail_lat': food_related_row[6], 'retail_lon': food_related_row[7],
                'geom': tripled_found.iloc[0]['geom']}
        
        data_to_append_list.append(data_to_append)
        

trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'lat', 'lon', 'retail_id', 'retail_lat', 'retail_lon', 'geom'])

In [9]:
trip_df

,user_id,trip_started_at,trip_finished_at,lat,lon,retail_id,retail_lat,retail_lon,geom
0,4,2022-09-06 13:13:38-04:00,2022-09-06 13:15:57-04:00,30.320775,-81.489335,217378,30.321608,-81.489418,LINESTRING (-81.4755320499999982 30.3195834200...
1,4,2022-09-09 13:13:30-04:00,2022-09-09 13:15:57-04:00,30.320775,-81.489347,217378,30.321608,-81.489418,LINESTRING (-81.4755399999999952 30.3195699999...
2,4,2022-09-12 13:13:05-04:00,2022-09-12 13:15:48-04:00,30.320777,-81.489343,217378,30.321608,-81.489418,LINESTRING (-81.4755399999999952 30.3195699999...
3,4,2022-09-13 21:16:06-04:00,2022-09-13 21:16:43-04:00,30.360139,-81.501590,117988,30.360104,-81.501938,LINESTRING (-81.5225891000000047 30.3481650000...
4,4,2022-09-21 14:26:08-04:00,2022-09-21 14:26:58-04:00,30.320821,-81.450708,56725,30.320976,-81.451164,LINESTRING (-81.4782170000000008 30.3177379999...
...,...,...,...,...,...,...,...,...,...
3464,12020,2022-10-28 22:29:39-04:00,2022-10-28 22:31:49-04:00,30.319412,-81.472914,51004,30.319002,-81.472450,LINESTRING (-81.4668631600000026 30.3378653500...
3465,12024,2022-10-20 18:18:21-04:00,2022-10-20 18:21:19-04:00,30.139440,-81.632660,140791,30.139462,-81.632706,LINESTRING (-81.6279990000000026 30.1753000000...
3466,12024,2022-10-25 18:51:08-04:00,2022-10-25 18:51:58-04:00,30.165053,-81.632655,169885,30.164610,-81.632446,LINESTRING (-81.6288700000000063 30.1726500000...
3467,12084,2022-10-21 15:16:44-04:00,2022-10-21 15:18:29-04:00,30.347375,-81.667600,124317,30.347931,-81.667053,LINESTRING (-81.6583499999999987 30.3489359999...


In [10]:
trip_df.to_csv('food_related_trip.csv')